In [28]:
!pip install -r requirements.txt -q

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [1]:
from ultralytics import YOLO
import cv2
import random
import numpy as np
import dotenv
import os
from inference_sdk import InferenceHTTPClient

In [2]:
from segmentator import Segmentator
from detector import Detector

In [3]:
segm = Segmentator(model_path="weights/segmentation.pt")
det = Detector(model_id="hip-joint-detection/2")

In [41]:
image1 = 'test_data/img1.jpg'
image2 = 'test_data/img2.jpg'
image3 = 'test_data/img3.jpg'
detected = 'test_data/detected.jpg'
segmented = 'test_data/segmented.jpg'

In [42]:
det.save_image(det.draw_detection(image3, det.detect(image3)), detected)
res_segm = segm.segment(detected)
segm.save_image(segm.draw_segmentation(detected,res_segm), "test_data/segmented.jpg")



0: 640x640 1 big-hole, 2 bone-hollows, 312.1ms
Speed: 2.4ms preprocess, 312.1ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


In [68]:
def find_tangent_lines(image_path, segmentation_results, detection_results):
    image = cv2.imread(image_path)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for result in segmentation_results:
        for seg in result.masks.xy:
            points = np.int32([seg])
            cv2.fillPoly(mask, points, 255)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No contours found in segmentation.")
        return image

    green_contour = max(contours, key=cv2.contourArea)

    small_holes = []
    for result in detection_results:
        if result['class'] == 'small-hole':
            x, y, w, h = result['x'], result['y'], result['width'], result['height']
            top_right = (int(x + w / 2), int(y - h / 2))
            top_left = (int(x - w / 2), int(y - h / 2))
            small_holes.append((top_left, top_right))

    if len(small_holes) < 2:
        print("Not enough 'small-hole' detections to draw tangent lines.")
        return image

    left_hole = min(small_holes, key=lambda x: x[0][0])
    right_hole = max(small_holes, key=lambda x: x[0][0])
    left_hole_top_left, left_hole_top_right = left_hole
    right_hole_top_left, right_hole_top_right = right_hole


    def line_intersects_contour(point1, point2, contour):
        mask_line = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.line(mask_line, point1, point2, 255, 1)
        mask_contour = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask_contour, [contour], -1, 255, 1)
        intersection = cv2.bitwise_and(mask_line, mask_contour)
        return np.any(intersection)
    

    left_line_points = None
    for angle in range(0, 90):
        length = 1000
        theta = np.radians(180 - angle)
        x_offset = int(length * np.cos(theta))
        y_offset = int(length * np.sin(theta))
        new_point = (left_hole_top_left[0] + x_offset, left_hole_top_left[1] - y_offset)
        if line_intersects_contour(left_hole_top_left, new_point, green_contour):
            left_line_points = (left_hole_top_left, new_point)
            cv2.line(image, left_hole_top_left, new_point, (255, 0, 0), 2)
            break

    right_line_points = None
    for angle in range(0, 90):
        length = 1000
        theta = np.radians(angle)
        x_offset = int(length * np.cos(theta))
        y_offset = int(length * np.sin(theta))
        new_point = (right_hole_top_right[0] + x_offset, right_hole_top_right[1] - y_offset)
        if line_intersects_contour(right_hole_top_right, new_point, green_contour):
            right_line_points = (right_hole_top_right, new_point)
            cv2.line(image, right_hole_top_right, new_point, (0, 255, 0), 2)
            break

    return image, left_line_points, right_line_points

In [70]:
res_segm = segm.segment(image1)
res_det = det.detect(image1)
# image = draw_tangent_lines(image1, res_segm, res_det)
_, left_line, right_line = find_tangent_lines(image1, res_segm, res_det)

image = cv2.imread(image1)
cv2.line(image, left_line[0], left_line[1], (255, 0, 0), 2)
cv2.line(image, right_line[0], right_line[1], (0, 255, 0), 2)
cv2.imwrite("test_data/tangent_lines.jpg", image)


0: 640x640 1 big-hole, 2 bone-hollows, 628.9ms
Speed: 21.3ms preprocess, 628.9ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)


True

In [15]:
res_detect2 = det.detect("test_data/img2.jpg")
res_detect3 = det.detect("test_data/img3.jpg")

In [28]:
def px_in_cm(results):
    pixels = None
    for res in results:
        if res['class'] == 'scale-mark':
            pixels = res['width']
            break
    return pixels * 0.8


In [30]:
for res in [res_detect, res_detect2, res_detect3]:
    pixels = px_in_cm(res)
    print(f"Pixels in cm: {pixels}")
    


Pixels in cm: 21.6
Pixels in cm: 20.0
Pixels in cm: 20.8
